In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e8/sample_submission.csv
/kaggle/input/playground-series-s5e8/train.csv
/kaggle/input/playground-series-s5e8/test.csv


In [2]:

# DATA MANIPULATION
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import math

# MACHINE LEARNING - SCIKIT-LEARN
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, RFE, RFECV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline

# ADVANCED ML LIBRARIES
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

# VISUALIZATION
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# CONFIGURATION FOR PLOTS
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# TEXT PROCESSING (NLP)
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud

# DEEP LEARNING 
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv2D, MaxPooling2D, Flatten
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    print("TensorFlow disponible")
except ImportError:
    print("TensorFlow no disponible en este entorno")


# UTILITIES
import os
import sys
import warnings
import itertools
from datetime import datetime, timedelta
import time
from collections import Counter
import pickle
import joblib

# JUPYTER SPECIFIC
from IPython.display import display, HTML
from tqdm.notebook import tqdm

# SUPPRESS WARNINGS
warnings.filterwarnings('ignore')

# PANDAS CONFIGURATION
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# NUMPY CONFIGURATION
np.random.seed(42)

# PLOTLY CONFIGURATION
pyo.init_notebook_mode(connected=True)

print("✅ Todas las librerías importadas correctamente!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")
print(f"🤖 Scikit-learn version: {__import__('sklearn').__version__}")
print(f"📈 Matplotlib version: {__import__('matplotlib').__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")


def quick_info(df):
    """Información rápida del dataset"""
    print(f"📊 Dataset Shape: {df.shape}")
    print(f"🔢 Columnas numéricas: {df.select_dtypes(include=[np.number]).columns.tolist()}")
    print(f"📝 Columnas categóricas: {df.select_dtypes(include=['object']).columns.tolist()}")
    print(f"❌ Valores nulos por columna:")
    print(df.isnull().sum()[df.isnull().sum() > 0])



2025-08-09 14:20:05.301352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754749205.583230      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754749205.668975      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow disponible


✅ Todas las librerías importadas correctamente!
📊 Pandas version: 2.2.3
🔢 NumPy version: 1.26.4
🤖 Scikit-learn version: 1.2.2
📈 Matplotlib version: 3.7.2
🎨 Seaborn version: 0.12.2


In [3]:
#https://www.kaggle.com/competitions/playground-series-s5e8/leaderboard? concurso
train = pd.read_csv("/kaggle/input/playground-series-s5e8/train.csv")
test =  pd.read_csv("/kaggle/input/playground-series-s5e8/test.csv")


https://www.kaggle.com/competitions/playground-series-s5e8/overview

*luego de cargar los datos , verificar si hay datos faltantes y tratar de hacer un randomforest lo mas rapido posible
para ver como viene el modelo y el leaderboard , despues profundizare para entender el dataset y ver como hacer
un feature eng y proceder a futuro


In [4]:
#age      : Age of the client (numeric)
#job      : Type of job (categorical: "admin.", "blue-collar", "entrepreneur", etc.)
#marital  : Marital status (categorical: "married", "single", "divorced")
#education: Level of education (categorical: "primary", "secondary", "tertiary", "unknown")
#default  : Has credit in default? (categorical: "yes", "no")
#balance  : Average yearly balance in euros (numeric)
#housing  : Has a housing loan? (categorical: "yes", "no")
#loan     : Has a personal loan? (categorical: "yes", "no")
#contact  : Type of communication contact (categorical: "unknown", "telephone", "cellular")
#day      : Last contact day of the month (numeric, 1-31)
#month    : Last contact month of the year (categorical: "jan", "feb", "mar", …, "dec")
#duration : Last contact duration in seconds (numeric)
#campaign : Number of contacts performed during this campaign (numeric)
#pdays    : Number of days since the client was last contacted from a previous campaign (numeric; -1 means the client was not previously contacted)
#previous : Number of contacts performed before this campaign (numeric)
#poutcome : Outcome of the previous marketing campaign (categorical: "unknown", "other", "failure", "success")
#y        : The target variable, whether the client subscribed to a term deposit (binary: "yes", "no")

In [5]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [6]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [7]:
quick_info(train)

📊 Dataset Shape: (750000, 18)
🔢 Columnas numéricas: ['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous', 'y']
📝 Columnas categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
❌ Valores nulos por columna:
Series([], dtype: int64)


In [8]:
quick_info(test)

📊 Dataset Shape: (250000, 17)
🔢 Columnas numéricas: ['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
📝 Columnas categóricas: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
❌ Valores nulos por columna:
Series([], dtype: int64)


In [9]:
#train["job"].value_counts()       # +12-1 management/blue-collar/technician/admin./services/retired/self-employed/entrepeneur/
                                  #       unemployed/housemaid/student/unknown
#train["marital"].value_counts()   # +3-1  married/single/divorced
#train["education"].value_counts() # +4-1 secondary/tertiary/primary/unknown
#train["default"].value_counts()   # binario reemplazo no = 0 (98.3%), yes  = 1 (1.7%)
#train["housing"].value_counts()   # binario reemplazo no = 0 (45.16%), yes  = 1 (54.83%)
#train["loan"].value_counts()      # binario reemplazo no = 0 (86.03%), yes  = 1 (13.97%)
#train["contact"].value_counts()   # one-hot 2 columnas , cellular/unknown/telephone si ambos son 0 entonces es unknown
#train["month"].value_counts()     # lo primero que pienso es ordinal pero hay mucha dif entre los valores
#                                  # la tasa de conversion de abajo me empuja a que sea one-hot o target encoding
#                                  # dado que mas tarde tmb quiero hacer una NN , hago one-hot y despues pruebo con Target
#month
#may    228411
#aug    128859
#jul    110647
#jun     93670
#nov     66062
#apr     41319
#feb     37611
#jan     18937
#oct      9204
#sep      7409
#mar      5802
#dec      2069
#train["poutcome"].value_counts() # one-hot 3 columnas unknown/failure/succes/other si todos son 0 entonces es unknown

In [10]:
# Calcular tasa de conversión por mes
conversion_by_month = train.groupby('month')['y'].agg(['count', 'sum', 'mean'])
conversion_by_month.columns = ['total_contacts', 'conversions', 'conversion_rate']
conversion_by_month = conversion_by_month.sort_values('conversion_rate', ascending=False)

print(conversion_by_month)

       total_contacts  conversions  conversion_rate
month                                              
mar              5802         3315         0.571355
sep              7409         3962         0.534755
dec              2069         1062         0.513291
oct              9204         4510         0.490004
apr             41319         9737         0.235654
feb             37611         7778         0.206801
jan             18937         2351         0.124148
aug            128859        14453         0.112161
nov             66062         7254         0.109806
jun             93670         9716         0.103726
jul            110647        10052         0.090847
may            228411        16298         0.071354


In [11]:
#pipeline One-hot
#para transformar train y test , para poder trabajar

class BankMarketingPreprocessor:
    def __init__(self):
        self.job_categories = None
        self.fitted = False
    
    def fit_transform(self, train_df):
        """
        Ajusta el preprocessor en los datos de entrenamiento y los transforma
        """
        df = train_df.copy()
        
        # 1. TARGET VARIABLE (ya viene como 0/1, no modificar)
        # df['y'] ya está en formato correcto
        
        # 2. VARIABLES BINARIAS (Label Encoding simple)
        binary_mappings = {
            'default': {'no': 0, 'yes': 1},
            'housing': {'no': 0, 'yes': 1}, 
            'loan': {'no': 0, 'yes': 1}
        }
        
        for col, mapping in binary_mappings.items():
            df[col] = df[col].map(mapping)
        
        # 3. EDUCATION (Ordinal)
        education_mapping = {
            'unknown': 0,
            'primary': 1, 
            'secondary': 2, 
            'tertiary': 3
        }
        df['education'] = df['education'].map(education_mapping)
        
        # 4. ONE-HOT ENCODING
        # Contact (3 categorías -> 2 columnas)
        df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
        df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        # unknown se infiere cuando ambas son 0
        
        # Month (12 categorías -> 11 columnas)
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
                 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        
        for month in months[1:]:  # Omitimos 'jan' como referencia
            df[f'month_{month}'] = (df['month'] == month).astype(int)
        
        # Poutcome (4 categorías -> 3 columnas)
        poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
        for outcome in poutcomes:
            df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
        # Job (12 categorías -> 11 columnas) - GUARDAR CATEGORÍAS PARA TEST
        self.job_categories = sorted(df['job'].unique())  # Ordenar para consistencia
        reference_job = self.job_categories[0]  # Primera categoría como referencia
        
        for job in self.job_categories:
            if job != reference_job:
                df[f'job_{job}'] = (df['job'] == job).astype(int)
        
        # Marital (3 categorías -> 2 columnas) 
        maritals = ['single', 'divorced']  # Omitimos 'married' como referencia
        for marital in maritals:
            df[f'marital_{marital}'] = (df['marital'] == marital).astype(int)
        
        # 5. MANTENER pdays ORIGINAL
        # No hacer nada con pdays - mantenerlo como está (-1 para no contactados)
        
        # 6. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS
        columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
        df = df.drop(columns=columns_to_drop)
        
        self.fitted = True
        return df
    
    def transform(self, test_df):
        """
        Transforma los datos de test usando los parámetros ajustados en train
        """
        if not self.fitted:
            raise ValueError("El preprocessor debe ser ajustado primero con fit_transform()")
        
        df = test_df.copy()
        
        # 1. VARIABLES BINARIAS (mismas que en train)
        binary_mappings = {
            'default': {'no': 0, 'yes': 1},
            'housing': {'no': 0, 'yes': 1}, 
            'loan': {'no': 0, 'yes': 1}
        }
        
        for col, mapping in binary_mappings.items():
            df[col] = df[col].map(mapping)
        
        # 2. EDUCATION (Ordinal - mismo que en train)
        education_mapping = {
            'unknown': 0,
            'primary': 1, 
            'secondary': 2, 
            'tertiary': 3
        }
        df['education'] = df['education'].map(education_mapping)
        
        # 3. ONE-HOT ENCODING (mismo que en train)
        # Contact
        df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
        df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        
        # Month (mismos meses que en train)
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
                 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        
        for month in months[1:]:  # Omitimos 'jan' como referencia
            df[f'month_{month}'] = (df['month'] == month).astype(int)
        
        # Poutcome (mismas categorías que en train)
        poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
        for outcome in poutcomes:
            df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
        # Job (USAR EXACTAMENTE LAS MISMAS CATEGORÍAS QUE EN TRAIN)
        if self.job_categories is None:
            raise ValueError("Las categorías de job no fueron guardadas durante fit_transform()")
        
        reference_job = self.job_categories[0]  # Misma referencia que en train
        
        for job in self.job_categories:
            if job != reference_job:
                df[f'job_{job}'] = (df['job'] == job).astype(int)
        
        # Marital (mismas categorías que en train)
        maritals = ['single', 'divorced']  # Omitimos 'married' como referencia
        for marital in maritals:
            df[f'marital_{marital}'] = (df['marital'] == marital).astype(int)
        
        # 4. MANTENER pdays ORIGINAL
        # No hacer nada con pdays - mantenerlo como está
        
        # 5. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS (mismas que en train)
        columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
        df = df.drop(columns=columns_to_drop)
        
        return df



In [12]:
#pipeline Target-Encoding
"""
class BankMarketingPreprocessorTargetEncoding:
    def __init__(self):
        self.target_encoders = {}
        self.fitted = False
    
    def fit_transform(self, train_df):

        df = train_df.copy()
        
        # 1. TARGET VARIABLE (ya viene como 0/1, no modificar)
        # df['y'] ya está en formato correcto
        
        # 2. VARIABLES BINARIAS (Label Encoding simple)
        binary_mappings = {
            'default': {'no': 0, 'yes': 1},
            'housing': {'no': 0, 'yes': 1}, 
            'loan': {'no': 0, 'yes': 1}
        }
        
        for col, mapping in binary_mappings.items():
            df[col] = df[col].map(mapping)
        
        # 3. EDUCATION (Ordinal)
        education_mapping = {
            'unknown': 0,
            'primary': 1, 
            'secondary': 2, 
            'tertiary': 3
        }
        df['education'] = df['education'].map(education_mapping)
        
        # 4. ONE-HOT ENCODING (solo para contact y poutcome)
        # Contact (3 categorías -> 2 columnas)
        df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
        df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        # unknown se infiere cuando ambas son 0
        
        # Poutcome (4 categorías -> 3 columnas)
        poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
        for outcome in poutcomes:
            df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
        # 5. TARGET ENCODING para job/marital/month
        
        # Job Target Encoding
        job_target_encoding = df.groupby('job')['y'].mean().to_dict()
        self.target_encoders['job'] = job_target_encoding
        df['job_target_encoded'] = df['job'].map(job_target_encoding)
        
        # Marital Target Encoding  
        marital_target_encoding = df.groupby('marital')['y'].mean().to_dict()
        self.target_encoders['marital'] = marital_target_encoding
        df['marital_target_encoded'] = df['marital'].map(marital_target_encoding)
        
        # Month Target Encoding
        month_target_encoding = df.groupby('month')['y'].mean().to_dict()
        self.target_encoders['month'] = month_target_encoding
        df['month_target_encoded'] = df['month'].map(month_target_encoding)
        
        # 6. MANTENER pdays ORIGINAL
        # No hacer nada con pdays - mantenerlo como está (-1 para no contactados)
        
        # 7. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS
        columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
        df = df.drop(columns=columns_to_drop)
        
        self.fitted = True
        return df
    
    def transform(self, test_df):

        if not self.fitted:
            raise ValueError("El preprocessor debe ser ajustado primero con fit_transform()")
        
        df = test_df.copy()
        
        # 1. VARIABLES BINARIAS (mismas que en train)
        binary_mappings = {
            'default': {'no': 0, 'yes': 1},
            'housing': {'no': 0, 'yes': 1}, 
            'loan': {'no': 0, 'yes': 1}
        }
        
        for col, mapping in binary_mappings.items():
            df[col] = df[col].map(mapping)
        
        # 2. EDUCATION (Ordinal - mismo que en train)
        education_mapping = {
            'unknown': 0,
            'primary': 1, 
            'secondary': 2, 
            'tertiary': 3
        }
        df['education'] = df['education'].map(education_mapping)
        
        # 3. ONE-HOT ENCODING (mismo que en train)
        # Contact
        df['contact_cellular'] = (df['contact'] == 'cellular').astype(int)
        df['contact_telephone'] = (df['contact'] == 'telephone').astype(int)
        
        # Poutcome (mismas categorías que en train)
        poutcomes = ['failure', 'other', 'success']  # Omitimos 'unknown' como referencia
        for outcome in poutcomes:
            df[f'poutcome_{outcome}'] = (df['poutcome'] == outcome).astype(int)
        
        # 4. TARGET ENCODING (usar valores calculados en train)
        
        # Job Target Encoding
        df['job_target_encoded'] = df['job'].map(self.target_encoders['job'])
        # Manejar valores no vistos (llenar con promedio general)
        global_job_mean = np.mean(list(self.target_encoders['job'].values()))
        df['job_target_encoded'] = df['job_target_encoded'].fillna(global_job_mean)
        
        # Marital Target Encoding
        df['marital_target_encoded'] = df['marital'].map(self.target_encoders['marital'])
        global_marital_mean = np.mean(list(self.target_encoders['marital'].values()))
        df['marital_target_encoded'] = df['marital_target_encoded'].fillna(global_marital_mean)
        
        # Month Target Encoding
        df['month_target_encoded'] = df['month'].map(self.target_encoders['month'])
        global_month_mean = np.mean(list(self.target_encoders['month'].values()))
        df['month_target_encoded'] = df['month_target_encoded'].fillna(global_month_mean)
        
        # 5. MANTENER pdays ORIGINAL
        # No hacer nada con pdays - mantenerlo como está
        
        # 6. ELIMINAR COLUMNAS ORIGINALES CATEGÓRICAS (mismas que en train)
        columns_to_drop = ['job', 'marital', 'contact', 'month', 'poutcome']
        df = df.drop(columns=columns_to_drop)
        
        return df
"""

'\nclass BankMarketingPreprocessorTargetEncoding:\n    def __init__(self):\n        self.target_encoders = {}\n        self.fitted = False\n    \n    def fit_transform(self, train_df):\n\n        df = train_df.copy()\n        \n        # 1. TARGET VARIABLE (ya viene como 0/1, no modificar)\n        # df[\'y\'] ya está en formato correcto\n        \n        # 2. VARIABLES BINARIAS (Label Encoding simple)\n        binary_mappings = {\n            \'default\': {\'no\': 0, \'yes\': 1},\n            \'housing\': {\'no\': 0, \'yes\': 1}, \n            \'loan\': {\'no\': 0, \'yes\': 1}\n        }\n        \n        for col, mapping in binary_mappings.items():\n            df[col] = df[col].map(mapping)\n        \n        # 3. EDUCATION (Ordinal)\n        education_mapping = {\n            \'unknown\': 0,\n            \'primary\': 1, \n            \'secondary\': 2, \n            \'tertiary\': 3\n        }\n        df[\'education\'] = df[\'education\'].map(education_mapping)\n        \n        

In [13]:
#target encoding
#preprocessor_encod = BankMarketingPreprocessorTargetEncoding()
               
#X_train_encod = preprocessor_encod.fit_transform(train)
#X_test_encod = preprocessor_encod.transform(test)

#y_train_encod_f = X_train_encod["y"]
#X_train_encod_f = X_train_encod("y", axis=1)

#print(f"Shape final train: {X_train_encod_f.shape}")
#print(f"Shape final test: {X_test_encod.shape}")
#print(f"Columnas finales: {list(X_train_encod_f.columns)}")
#print(f"Columnas finales: {list(X_test_encod.columns)}")

In [14]:
preprocessor = BankMarketingPreprocessor()
               
X_train_p = preprocessor.fit_transform(train)
X_test_p = preprocessor.transform(test)

y_train_f = X_train_p["y"]
X_train_f = X_train_p.drop("y", axis=1)

print(f"Shape final train: {X_train_f.shape}")
print(f"Shape final test: {X_test_p.shape}")
print(f"Columnas finales: {list(X_train_f.columns)}")
print(f"Columnas finales: {list(X_test_p.columns)}")

Shape final train: (750000, 41)
Shape final test: (250000, 41)
Columnas finales: ['id', 'age', 'education', 'default', 'balance', 'housing', 'loan', 'day', 'duration', 'campaign', 'pdays', 'previous', 'contact_cellular', 'contact_telephone', 'month_feb', 'month_mar', 'month_apr', 'month_may', 'month_jun', 'month_jul', 'month_aug', 'month_sep', 'month_oct', 'month_nov', 'month_dec', 'poutcome_failure', 'poutcome_other', 'poutcome_success', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'marital_single', 'marital_divorced']
Columnas finales: ['id', 'age', 'education', 'default', 'balance', 'housing', 'loan', 'day', 'duration', 'campaign', 'pdays', 'previous', 'contact_cellular', 'contact_telephone', 'month_feb', 'month_mar', 'month_apr', 'month_may', 'month_jun', 'month_jul', 'month_aug', 'month_sep', 'month_oct', 'month_nov', 'month_dec', 'pout

In [15]:
X_train_p.head()

,id,age,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,contact_cellular,contact_telephone,month_feb,month_mar,month_apr,month_may,month_jun,month_jul,month_aug,month_sep,month_oct,month_nov,month_dec,poutcome_failure,poutcome_other,poutcome_success,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_single,marital_divorced
0,0,42,2,0,7,0,0,25,117,3,-1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,38,2,0,514,0,0,18,185,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,36,2,0,602,1,0,14,111,2,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,3,27,2,0,34,1,0,28,10,2,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,4,26,2,0,889,1,0,3,902,1,-1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [16]:
X_test_p.head()

,id,age,education,default,balance,housing,loan,day,duration,campaign,pdays,previous,contact_cellular,contact_telephone,month_feb,month_mar,month_apr,month_may,month_jun,month_jul,month_aug,month_sep,month_oct,month_nov,month_dec,poutcome_failure,poutcome_other,poutcome_success,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_single,marital_divorced
0,750000,32,2,0,1397,1,0,21,224,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,750001,44,3,0,23,1,0,3,586,2,-1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,750002,36,1,0,46,1,1,13,111,2,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,750003,58,2,0,-1380,1,1,29,125,1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,750004,28,2,0,1950,1,0,22,181,1,-1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [17]:
# Guardar IDs antes de eliminar
train_ids = X_train_f['id'].copy()
test_ids = X_test_p['id'].copy()

# Eliminar ID de los datasets
X_train_final = X_train_f.drop('id', axis=1)  # 40 columnas
X_test_final = X_test_p.drop('id', axis=1)    # 40 columnas

print(f"Train shape sin ID: {X_train_final.shape}")
print(f"Test shape sin ID: {X_test_final.shape}")

Train shape sin ID: (750000, 40)
Test shape sin ID: (250000, 40)


In [18]:
%%time
xgb_model2 = xgb.XGBClassifier(
        n_estimators=475,
        learning_rate=0.0718,
        max_depth=9,
        subsample=0.8006,
        colsample_bytree=0.8169,
        colsample_bylevel=0.7323,
        min_child_weight=7,
        gamma=0.4794,
        reg_alpha=0.6923,
        reg_lambda=0.9114,
        random_state=42,
        n_jobs=-1,
        eval_metric='auc',
        use_label_encoder=False,
        verbosity=0
)



# Entrenar
xgb_model2.fit(X_train_final, y_train_f)

# Hacer predicciones
y_test_pred = xgb_model2.predict_proba(X_test_final)[:, 1]

# Submission
submission = pd.DataFrame({
    'id': test_ids,
    'y': y_test_pred
})
submission.to_csv('submission.csv', index=False)

submission.head()

CPU times: user 2min 11s, sys: 436 ms, total: 2min 11s
Wall time: 34.6 s


,id,y
0,750000,0.001528
1,750001,0.052252
2,750002,0.000208
3,750003,0.000062
4,750004,0.014794


In [19]:
#v1 150 iteraciones  CV Score: 0.9613 en competencia 0.89948 ,el primero tiene 0.97778 7/8/2025
#rf1 = RandomForestClassifier( #rf target encod 150 iter             
#    n_estimators=100,                   #
#    min_samples_split=500,              #
#    min_samples_leaf=200,               #
#    max_features=0.3,                   #
#    max_depth= None,                    #
#    criterion = "entropy",              #
#    bootstrap = False,                  #
#    random_state=42,                    #
#    n_jobs=-1,                          #
#    class_weight='balanced'             #
#    )
"""=== FEATURE IMPORTANCE (TOP 10) ===
             feature  importance
7           duration    0.647334
3            balance    0.082880
26  poutcome_success    0.040380
11  contact_cellular    0.038691
4            housing    0.037123
9              pdays    0.020095
0                age    0.016717
6                day    0.014675
14         month_mar    0.013597
10          previous    0.009041
"""
#v2 XGB 250 iteraciones CV Score: 0.9676 en competencia 0.96850
#xgb_model1 = xgb.XGBClassifier(
#        n_estimators=366,              #
#        learning_rate=0.1316,          #
#        max_depth=8,                   #
#        subsample=0.8724,              #
#        colsample_bytree=0.7738,       #
#        colsample_bylevel=0.7321,      #
#        min_child_weight=4,            #
#        gamma=0.1268,                  #
#        reg_alpha=0.8228,              #
#        reg_lambda=0.7442,             #
#        random_state=42,               #
#        n_jobs=-1,                     #
#        eval_metric='auc',             #
#        use_label_encoder=False,       #
#        verbosity=0                    #
#) 
"""
target Encoding v1 rfe
=== MEJORES PARÁMETROS ===
{'n_estimators': 200, 'min_samples_split': 100, 'min_samples_leaf': 50, 'max_features': 0.3, 'max_depth': 20, 'criterion': 'gini',
 'bootstrap': False}

=== MEJOR SCORE (AUC) ===
CV Score: 0.9620

=== FEATURE IMPORTANCE (TOP 10) ===
                 feature  importance
7               duration    0.651461
3                balance    0.076133
18  month_target_encoded    0.071657
15      poutcome_success    0.038616
11      contact_cellular    0.034448
4                housing    0.033808
6                    day    0.017222
9                  pdays    0.015846
0                    age    0.013047
16    job_target_encoded    0.011721
"""

#v3 xgb 2500 iteraciones One hot y xgb 2500 iteracion Target Encod , CV Score: 0.9676 en One-hot y target, en competencia 0.96850
# si o si ver de feature eng y/o una NN para continuar
#xgb_model2 = xgb.XGBClassifier(
#        n_estimators=475,          # 
#        learning_rate=0.0718,      #
#        max_depth=9,               #
#        subsample=0.8006,          #
#        colsample_bytree=0.8169,   #
#        colsample_bylevel=0.7323,  #
#        min_child_weight=7,        #
#        gamma=0.4794,              #
#        reg_alpha=0.6923,          #
#        reg_lambda=0.9114,         #
#        random_state=42,           #
#        n_jobs=-1,                 #
#        eval_metric='auc',         #
#        use_label_encoder=False,   #
#        verbosity=0                #
#)
#Mejor score (CV): 0.9678

"""
#2500 Target
Mejores parámetros:
{'colsample_bylevel': np.float64(0.7502648014537899), 'colsample_bytree': np.float64(0.7346954904208862), 'gamma': np.float64(0.2506071839698449),
 'learning_rate': np.float64(0.0896867318135531), 'max_depth': 9, 'min_child_weight': 4, 'n_estimators': 424,
 'reg_alpha': np.float64(0.6156748663350115), 'reg_lambda': np.float64(0.8680923960240279), 'subsample': np.float64(0.9350496976685249)}

Mejor score (CV): 0.9678

"""

"\n#2500 Target\nMejores parámetros:\n{'colsample_bylevel': np.float64(0.7502648014537899), 'colsample_bytree': np.float64(0.7346954904208862), 'gamma': np.float64(0.2506071839698449),\n 'learning_rate': np.float64(0.0896867318135531), 'max_depth': 9, 'min_child_weight': 4, 'n_estimators': 424,\n 'reg_alpha': np.float64(0.6156748663350115), 'reg_lambda': np.float64(0.8680923960240279), 'subsample': np.float64(0.9350496976685249)}\n\nMejor score (CV): 0.9678\n\n"